# 타이타닉 생존자 예측

In [21]:
import seaborn as sns
import numpy as np
import pandas as pd

In [22]:
df = sns.load_dataset('titanic')
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


- ##### 데이터 전처리

In [28]:
# Feature selection
df = df[['survived','pclass','sex','age','sibsp','parch','fare','embarked','deck']]
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN
3,1,1,female,35.0,1,0,53.1000,S,C
4,0,3,male,35.0,0,0,8.0500,S,NaN


In [24]:
# Nan처리
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [25]:
# 나이는 평균으로 대체
df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

0

In [26]:
# embarked는 최빈값(가장많은 것)으로 대체
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [35]:
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [36]:
# deck는 결측치가 매우 많으므로 삭제
df.drop(columns=['deck'], inplace=True)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


In [37]:
# sex, embarked를 숫자로 변환
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['sex'] = encoder.fit_transform(df.sex)
df['embarked'] = encoder.fit_transform(df.embarked)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


- ##### Train/Test dataset으로 분리

In [60]:
X = df.iloc[:,1:].values
y = df['survived'].values
X.shape, y.shape

((891, 7), (891,))

In [68]:
df.iloc[:,:1].values.shape

(891, 1)

In [61]:
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y , test_size=0.2, random_state=2021 , stratify=y, 
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

- ##### RandomForest 모델 생성 및 학습

In [63]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [64]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2021)

In [69]:
rfc.score(X_test, y_test)

0.8100558659217877

- ##### GridSearchCV 수행

In [72]:
params = {
    'max_depth' : [3,4,5,6,7],
    'min_samples_split' : [2,3,4]
}

In [73]:
from sklearn.model_selection import GridSearchCV

In [74]:
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=3)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [75]:
grid_rf.best_params_

{'max_depth': 7, 'min_samples_split': 2}

In [76]:
params = {
    'max_depth' : [6,7,8,9],
    'min_samples_split' : [2,3,4]
}

In [77]:
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=3)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [78]:
grid_rf.best_params_

{'max_depth': 7, 'min_samples_split': 2}

In [79]:
grid_rf.score(X_test, y_test)

0.8324022346368715

- ##### 테스트 데이터에 적용

In [80]:
y_test[100],X_test[100]

(1, array([ 1.,  1., 31.,  1.,  0., 57.,  2.]))

In [85]:
grid_rf.predict(X_test[100].reshape(1,-1))              # reshape하는 이유는 X_test는 2차원
                                                        # x_test[100]은 1차원

array([0], dtype=int64)

- ##### 타이타닉 엉터리 분류기

In [87]:
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    # fit(), predict() method는 재정의(overriding)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            if X[i,1] == 0 :            # 0은 여성, 여성이면 생존으로 처리
                pred[i,0] = 1
        return pred

In [91]:
my_clf = MyClassifier()
my_clf.fit(X_train, y_train)
pred = my_clf.predict(X_test)

In [92]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.7877094972067039